In [2]:
import zmq
import datetime
import pandas as pd
import numpy as np
import numpy
from numpy import inf

import json
import plotly_stream as plyst
import plotly.tools as plyt
import plotly.plotly as ply
#!pip install plotly
import tpqib
import datetime

#from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import matplotlib.pyplot as plt

import pickle

iterations = 0
df = pd.DataFrame()
pdf= pd.DataFrame()
final=pd.DataFrame()

context = zmq.Context()

#Forwarding ML output
socket_pub = context.socket(zmq.PUB)
socket_pub.bind('tcp://127.0.0.1:7010')

port = "7000"
# socket to talk to server
socket_sub = context.socket(zmq.SUB)
print ("Collecting & plotting stock prices.")
socket_sub.connect("tcp://localhost:%s" % port)

socket_sub.setsockopt_string(zmq.SUBSCRIBE, u'SPY')


In [6]:
%run preprocessing.py

In [7]:
%run km_arima_lstm.py

In [9]:
'''
## warm up upto preprocessing
#final=pd.DataFrame()
#window=100
#for _ in range(window):
while True:
    iterations += 1
    string = socket_sub.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'Stock':sym,'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing_df(df)
    x = data.to_string(header=False,index=False,index_names=False).split('\n')
    #print(df.tail(1))
    #print(data.tail(1))
    socket_pub.send_string(x[-1])
    print(x[-1]) 
'''    

"\n## warm up upto preprocessing\n#final=pd.DataFrame()\n#window=100\n#for _ in range(window):\nwhile True:\n    iterations += 1\n    string = socket_sub.recv_string()\n    sym, bidPrice,bidSize,askPrice,askSize = string.split()\n    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))\n    dt = datetime.datetime.now()\n    df = df.append(pd.DataFrame({'Stock':sym,'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))\n    data=preprocessing_df(df)\n    x = data.to_string(header=False,index=False,index_names=False).split('\n')\n    #print(df.tail(1))\n    #print(data.tail(1))\n    socket_pub.send_string(x[-1])\n    print(x[-1]) \n"

In [ ]:
## warm up upto preprocessing
#final=pd.DataFrame()

#window=10
#for _ in range(window):
while True:
    iterations += 1
    string = socket_sub.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'Stock':sym,'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing_df(df)
    
    pre_arima=ARIMA_df(data.logDiff)
    data['pr_arima']=np.exp(float(pre_arima[-1])+df.log.shift(60))
    
    data=kalman_ma(data)
    data=strat_lr(data)
    data=classification_up_dn(data)
    
    data['predict_svm']=data.predict_svm_up+data.predict_svm_dn
    data['predict_lm']=data.predict_lm_up+data.predict_lm_dn
    #data['UD']=np.where(data.predict_svm+data.predict_lm>0,1,np.where(data.predict_svm+data.predict_lm<0,-1,0))  
    data['UD']=np.where(np.logical_and(data.predict_svm>0,data.predict_lm>0),1,np.where(np.logical_and(data.predict_svm<0,data.predict_lm<0),-1,0))  
    #data=data.dropna()   
    data['spread']=data.Close-data.pr_arima

    df_LSTM= data[['askPrice','askSize','bidPrice','bidSize','Close','spread','pr_arima','sigma']]
    pr,y=strat_LSTM(df_LSTM)
    UD=pr[-1]-y[-1]
    final=final.append(pd.DataFrame({'LSTM':float(UD)},index=[dt]))
    #output=data[['Close','price','km','pr_arima','pREG','pSVR','UD']]
    output=data[['Stock','Close','price','km','pr_arima','UD']]
    #output['spread']=data.pREG-data.SVR
    output['LSTM']=final.LSTM
    
    
    #print(df.tail(1))
    #print(data.tail(1))
    #print(output.tail(1))
       
    
    #x = data.to_string(header=False,index=False,index_names=False).split('\n')
    #socket_pub.send_string(x[-1])
    #print(x[-1]) 
    
    x = output.to_string(header=False,index=False,index_names=False).split('\n')
    socket_pub.send_string(x[-1])
    print(x[-1]) 


/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/octo/Dropbox/trading_system/system_sep/km_arima_lstm.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

SPY  249.725  249.720313  249.727817  249.730002  1  0.362725
SPY  249.725  249.720252  249.726936  249.730002  1  0.363438
SPY  249.725  249.720584  249.726174  249.725002  1  0.292169
SPY  249.725  249.722670  249.725681  249.725002  1  0.277042
SPY  249.725  249.721328  249.725108  249.725002  1  0.210131
SPY  249.725  249.720697  249.724530  249.725002  1  0.192468
SPY  249.725  249.720682  249.724007  249.725002  1  0.190306
SPY  249.725  249.720398  249.723508  249.725002  1  0.195557
SPY  249.725  249.720423  249.723057  249.730002  1  0.194891
SPY  249.725  249.720398  249.722646  249.730002  1  0.194546
SPY  249.725  249.720126  249.722247  249.735002  1  0.264827
SPY  249.725  249.720128  249.721884  249.735002  1  0.279546
SPY  249.725  249.720494  249.721591  249.735002  1  0.346632
SPY  249.725  249.720511  249.721328  249.735002  1  0.369909
SPY  249.725  249.720776  249.721116  249.735002  1  0.367100
SPY  249.725  249.720765  249.720922  249.740002  1  0.246367
SPY  249

In [18]:
import pickle
#from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


df=data.dropna()
# saving linear model
df=df[1:]
X=df[['askPrice','askSize','bidPrice','bidSize','Close','U','D','sigma']]
y=df[['logDiff']]
regr = linear_model.LinearRegression()
regr_model=regr.fit(X,y)
regr_model = pickle.dumps(regr_model)
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.9) #kernel='linear' #kernel='poly'
svr_model = svr_rbf.fit(X, y)
svr_model = pickle.dumps(svr_model)

# saving logistics and SVC model
df=df[1:].dropna()
X=df[['askPrice','askSize','bidPrice','bidSize','Close','price','sigma']]
y1=df[['U']]
y2=df[['D']]

svm = SVC(kernel='linear')
lm = linear_model.LogisticRegression(C=1e4)
svm_model_up= svm.fit(X,y1)
svm_model_up = pickle.dumps(svm_model_up)
lm_model_up= lm.fit(X, y1)
lm_model_up = pickle.dumps(lm_model_up)
svm_model_dn= svm.fit(X, y2)
svm_model_dn = pickle.dumps(svm_model_dn)
lm_model_dn= lm.fit(X, y2)
lm_model_dn = pickle.dumps(lm_model_dn)


#loading regression model, first save the model
svr_model = pickle.loads(svr_model)
regr_model = pickle.loads(regr_model)

#loading classification model, first save the model
svm_model_up = pickle.loads(svm_model_up)
svm_model_dn = pickle.loads(svm_model_dn)
lm_model_up = pickle.loads(lm_model_up)
lm_model_dn = pickle.loads(lm_model_dn)

def strat_lr(data):
    data=data.tail(60).dropna()
    X=data[['askPrice','askSize','bidPrice','bidSize','Close','U','D','sigma']]
    y=data[['logDiff']]
    predict_regr=regr_model.predict(X)
    predict_svr=svr_model.predict(X)
    dt=data[['Close']]
    dt['predict_regr']=predict_regr
    dt['predict_svr']=predict_svr
        
    #pdf=data
    #pdf['pREG']=np.exp(dt.predict_regr+data.log.shift(59))
    #pdf['pSVR']=np.exp(dt.predict_regr+data.log.shift(59))
    data['pREG']=np.exp(dt.predict_regr+data.log.shift(59))
    data['pSVR']=np.exp(dt.predict_regr+data.log.shift(59))
    return data

def classification_up_dn(data):
    X=data[['askPrice','askSize','bidPrice','bidSize','Close','price','sigma']]
    y1=data[['U']]
    y2=data[['D']]
    pr_df=data.tail(60)
    predict_svm_up=svm_model_up.predict(X.tail(60))
    pr_df['predict_svm_up']=predict_svm_up
    predict_lm_up=lm_model_up.predict(X.tail(60))
    pr_df['predict_lm_up']=predict_lm_up
    predict_svm_dn=svm_model_dn.predict(X.tail(60))
    pr_df['predict_svm_dn']=predict_svm_dn
    predict_lm_dn=lm_model_dn.predict(X.tail(60))
    pr_df['predict_lm_dn']=predict_lm_dn
    pr_df['predict_svm']=pr_df.predict_svm_up+pr_df.predict_svm_dn
    pr_df['predict_lm']=pr_df.predict_lm_up+pr_df.predict_lm_dn
    return pr_df

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

